# Visualize the data

This paper used a reverse substitution method to generate the dataset as follows: 

A previous work provided a list of medical acronyms and their corresponding expansions, this formed the key-value pairs of targets. The authors then secured medical notes and identified sentences that contained expansions that are present in the targets dataset. These expansions were replaced in-place by the acronym that was present in the targets database. The data therefore contains the following fields:

- id
- text: the full source sentence
- location: the word index where the acronym was present
- label: the correct expansion of the acronym
- label_num: the index of the expansion in the label field. Since we are training a classification model, this index is what is actually outputted by the model, and is used to calculate the loss (generated by the load_dataframes function)

In [7]:
import pandas as pd 
import os

from utils import load_dataframes, load_model, train_loop

import torch
from transformers import ElectraTokenizer
from models.tokenizer_and_dataset import HuggingfaceDataset

pd.set_option('display.max_colwidth', None)
DEVICE = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")

data_path = "data/train/data.csv"
adam_path = "toy_data/valid_adam.txt" # this contains a list of all valid expansions that were used in reverse substution (from this paper: https://academic.oup.com/bioinformatics/article/22/22/2813/197656)

In [3]:
train_data = pd.read_csv(data_path, nrows=300)

In [4]:
adam_df = pd.read_csv(adam_path, sep='\t')
len(adam_df.EXPANSION.unique())
adam_df.head()

,PREFERRED_AB,VARIANT_AB,EXPANSION
0,+dP/dt,+dP/dt:18|+dp/dt:3,contractility
1,+dP/dt,+dP/dt:6,rate of left ventricular pressure development
2,"1,5-AG","1,5-AG:10|1,5AG:3",anhydrodglucitol
3,"1,5-AG","1,5-AG:14|1,5AG:11",anhydroglucitol
4,14C,14C:8,labeled with radioactive carbon


In [5]:
train, valid, test, label_to_ix = \
        load_dataframes(data_dir="data", data_filename="data.csv", adam_path=adam_path)

train.head()

,ABSTRACT_ID,TEXT,LOCATION,LABEL,LABEL_NUM
0,14145090,velvet antlers vas are commonly used in traditional chinese medicine and invigorant and contain many PET components for health promotion the velvet antler peptide svap is one of active components in vas based on structural study the svap interacts with tgfÎ² receptors and disrupts the tgfÎ² pathway we hypothesized that svap prevents cardiac fibrosis from pressure overload by blocking tgfÎ² signaling SDRs underwent TAC tac or a sham operation T3 one month rats received either svap mgkgday or vehicle for an additional one month tac surgery induced significant cardiac dysfunction FB activation and fibrosis these effects were improved by treatment with svap in the heart tissue tac remarkably increased the expression of tgfÎ² and connective tissue growth factor ctgf ROS species C2 and the phosphorylation C2 of smad and ERK kinases erk svap inhibited the increases in reactive oxygen species C2 ctgf expression and the phosphorylation of smad and erk but not tgfÎ² expression in cultured cardiac fibroblasts angiotensin ii ang ii had similar effects compared to tac surgery such as increases in Î±smapositive CFs and collagen synthesis svap eliminated these effects by disrupting tgfÎ² IB to its receptors and blocking ang iitgfÎ² downstream signaling these results demonstrated that svap has antifibrotic effects by blocking the tgfÎ² pathway in CFs,63,transverse aortic constriction,20135
1,1900667,the clinical features of our cases demonstrated some of the already known characteristics of the variable spectrum of hiv infection da are the most important risk category in italy of the arc cases evolved into aids during a month followup on average the most frequent oi in our aids cases were pcp c albicans esophagitis and chronic mucocutaneous ulcers an high percentage of neurologic involvement from hiv was observed and HM were encountered in aids ks and undifferentiated b lymphoma as well as in arc HD statistically significant worsening of the immunologic situation is evident as the disease progresses from las to aids G1 b lymphocytes represent most of the cells of the germinal center during the hyperplastic stage of lymphadenopathy reversal of the tt ratio appears early during the initial stage of lymphadenopathy and is due to a decrease of cd and a relative increase of cd also destruction of the follicular dendritic cells is an early feature which becomes more evident as the disease advances and the lymph node evolves toward progressive involution activated blymphocyte augmentation with polyclonal ig secretion appears to be related to TI b stimulation by coinfection such as cmv ebv and hbv the increase of CD8 lymphocytes seems to be partly related to the excessive activation of b lymphocytes and partially directed to the cells INF by hiv or coated with its proteins the destruction of follicular dendritic cells has been interpreted not only as a killer effect of the virus but also as a result of the intervention of ctl sensitized to the cells containing the virus their destruction may contribute to the impaired recognition of soluble antigen which is one of the main features of the immune deficiency of hiv infection,85,hodgkins lymphoma,9002
2,8625554,ceftobiprole bpr is an investigational cephalosporin with activity against staphylococcus aureus including methicillinresistant s aureus mrsa strains the pharmacodynamic pd profile of bpr against s aureus strains with a variety of susceptibility phenotypes in an immunocompromised murine pneumonia model was characterized the bpr mics of the test isolates ranged from to mugml pharmacokinetic pk studies were conducted with infected neutropenic balbc mice and the bpr concentrations were measured in plasma epithelial lining fluid elf and lung tissue pd studies with these mice were undertaken with eight s aureus isolates two MSSA strains three hospitalacquired mrsa strains and three CA mrsa strains subcutaneous bpr doses of to mgkg of body weightday

# Tokenization

Now we have our train, test, and validation data. The sentences need to be tolkenized - broken down into chunks that are manageble yet meaningful. There are three ways we can break down a sentence:
- word-level (unigrams, bigrams, etc). This  makes the vocabulary (list of all possible tolkens) extremely large since it must cover all possible words. The training data may not even have examples of all word-level tolkens
- char-level (a, b, c, etc.). The vocab is nice and small (26 chars in english) but these tolkens hold little information alone.
- sub-word-level (token, izat, ion). Strikes a middle ground where the vocab size is managable, yet each token still holds some meaning

In the paper the authors use the pre-build electra-tolkenizer, which uses [WordPiece tokenization](https://huggingface.co/course/chapter6/6?fw=pt) under-the-hood. This is a sub-word style tokenizer that determines tokens based on the probability of different character pairs appearing in the training text.


In [8]:
# Create tokenizer objects
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')

# Create torch Dataset objects
train_data = HuggingfaceDataset(train, tokenizer=tokenizer, device=DEVICE)
valid_data = HuggingfaceDataset(valid, tokenizer=tokenizer, device=DEVICE)